# Combining Datasets with Pandas 

![pandas](http://tkgenius.com/wp-content/uploads/2012/04/happy-panda-300x188.png)

Pandas functions that allow us to combine two sets of data include the use of `pd.merge()`, `pd.concat()`, `df.join()`, and `df.merge()`. For the most part, these four do largely the same things (although you'll notice the slight syntax difference between the first two being called on the `pandas` module and the second two being called on a `DataFrame` object). There are some cases where one of these might be better than another in terms of writing less code or performing some kind of data combination in an easier way. The major differences between these, though, largely depend on what they do by default when you try to combine different data. By default, `merge()` looks to join on common columns, `join()` on common indices, and `concat()` by just appending on a given axis.

You can find more detail about the differences between all three of these in the [docs](http://pandas.pydata.org/pandas-docs/stable/merging.html). We'll look at some examples below. 

## Why does this matter?

In short, this doesn't actually matter too much (to be honest). This is really just a notebook to give you an idea of the differences between the different data combining methods available with DataFrames. The one takeaway from this notebook might be the difference between `concat()` and `append()` towards the end of this notebook. There is actually a use case when we are forced to use `concat` instead of `append`. 

In [1]:
import pandas as pd
white_wine_df = pd.read_csv('data/winequality-white.csv', delimiter=';')
red_wine_df = pd.read_csv('data/winequality-red.csv', delimiter=';')

In [2]:
# Let's check out the columns before playing around. 
print white_wine_df.columns
print red_wine_df.columns

Index([u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol',
       u'quality'],
      dtype='object')
Index([u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol',
       u'quality'],
      dtype='object')


In [3]:
# It looks like quality might be categorical - let's check it's unique values to see if it is. 
print white_wine_df.quality.unique()
print red_wine_df.quality.unique()

[6 5 7 8 4 3 9]
[5 6 7 4 8 3]


## Looking at different Methods 

Let's say that I wanted to get the average pH and alcohol by quality for each type of wine into a single DataFrame. Let's walk through that using both `join` and `merge`. We'll start by looking at `join`. 

In [4]:
# The first thing I'm going to need to do is actually get the average pH and alcohol for 
# each quality category for the wines. 
whites_avg_pH = white_wine_df.groupby('quality')[['pH', 'alcohol']].mean()
reds_avg_pH = red_wine_df.groupby('quality')[['pH', 'alcohol']].mean()

In [5]:
# Notice these give me back DataFrames (If I had only asked for one column right above, e.g. 
# pH instead of pH **and** alcohol, I would have gotten back Series). 
print type(whites_avg_pH)
print type(reds_avg_pH)

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [6]:
# Also note that the DataFrames are now indexed by what we grouped by above. 
print whites_avg_pH.index
print reds_avg_pH.index

Int64Index([3, 4, 5, 6, 7, 8, 9], dtype='int64', name=u'quality')
Int64Index([3, 4, 5, 6, 7, 8], dtype='int64', name=u'quality')


In [7]:
print 'Reds'
print '-' * 50
print reds_avg_pH 
print 

print 'Whites'
print '-' * 50
print whites_avg_pH

Reds
--------------------------------------------------
               pH    alcohol
quality                     
3        3.398000   9.955000
4        3.381509  10.265094
5        3.304949   9.899706
6        3.318072  10.629519
7        3.290754  11.465913
8        3.267222  12.094444

Whites
--------------------------------------------------
               pH    alcohol
quality                     
3        3.187500  10.345000
4        3.182883  10.152454
5        3.168833   9.808840
6        3.188599  10.575372
7        3.213898  11.367936
8        3.218686  11.636000
9        3.308000  12.180000


### Using Join

In [8]:
# Let's start off with joining. Remember joins combine on commmon indices by default. 
reds_avg_pH.join(whites_avg_pH)

ValueError: columns overlap but no suffix specified: Index([u'pH', u'alcohol'], dtype='object')

In [9]:
# What's this error? Well, if you're columns overlap, it doesn't know which ones to keep, and 
# rather than making that decision for you, they tell you you need to specify how to label 
# the columns coming from each DF (so here we have to label them with a suffix). Another option
# would just be to rename our columns (this depends on your ultimate goal at the end). 
reds_avg_pH.join(whites_avg_pH, lsuffix = '_reds', rsuffix='_whites')

,pH_reds,alcohol_reds,pH_whites,alcohol_whites
quality,,,,
3,3.398000,9.955000,3.187500,10.345000
4,3.381509,10.265094,3.182883,10.152454
5,3.304949,9.899706,3.168833,9.808840
6,3.318072,10.629519,3.188599,10.575372
7,3.290754,11.465913,3.213898,11.367936
8,3.267222,12.094444,3.218686,11.636000


In [10]:
# Okay, so we lost the white wines with quality 9 since join performs an inner join by default. 
# I really like my high quality white wines, so let's figure out how to get those in. 
reds_avg_pH.join(whites_avg_pH, how = 'right', lsuffix='_reds', rsuffix='_whites')

,pH_reds,alcohol_reds,pH_whites,alcohol_whites
quality,,,,
3,3.398000,9.955000,3.187500,10.345000
4,3.381509,10.265094,3.182883,10.152454
5,3.304949,9.899706,3.168833,9.808840
6,3.318072,10.629519,3.188599,10.575372
7,3.290754,11.465913,3.213898,11.367936
8,3.267222,12.094444,3.218686,11.636000
9,NaN,NaN,3.308000,12.180000


### Using Merge

Now we'll move on to looking at how to do this with `merge` (both on a `DataFrame` object and using the `pd` module). It turns out that using `join` actually calls `merge` under the hood, and so which one you choose really just depends on whatever seems easiest, or allows you to write the least amount of clean, readable code (or that works, if you for some reason can't get one of the two to work). 

From the Pandas [docs](http://pandas.pydata.org/pandas-docs/stable/merging.html): 

The related DataFrame.join method, uses merge internally for the index-on-index and index-on-column(s) joins, but joins on indexes by default rather than trying to join on common columns (the default behavior for merge). If you are joining on index, you may wish to use DataFrame.join to save yourself some typing.

In [11]:
reds_avg_pH.merge(whites_avg_pH)

,pH,alcohol


Since `merge` combines on common columns by default, we might expect the above to work. After all, we know that both of these DataFrames have a `pH` column and an `alcohol` column. So why didn't it? It's because no pair of `[pH, alochol]` combo. had the same values in each of the `DataFrames`, so they had nothing in common to merge on. So sad...

In [12]:
# Okay, let's reset the index of both of our DataFrames to then have a common column to 
# merge on. 
reds_avg_pH_reset = reds_avg_pH.reset_index()
whites_avg_pH_reset = whites_avg_pH.reset_index()

In [13]:
print 'Reds'
print '-' * 50
print reds_avg_pH_reset
print 

print 'Whites'
print '-' * 50
print whites_avg_pH_reset

Reds
--------------------------------------------------
   quality        pH    alcohol
0        3  3.398000   9.955000
1        4  3.381509  10.265094
2        5  3.304949   9.899706
3        6  3.318072  10.629519
4        7  3.290754  11.465913
5        8  3.267222  12.094444

Whites
--------------------------------------------------
   quality        pH    alcohol
0        3  3.187500  10.345000
1        4  3.182883  10.152454
2        5  3.168833   9.808840
3        6  3.188599  10.575372
4        7  3.213898  11.367936
5        8  3.218686  11.636000
6        9  3.308000  12.180000


In [14]:
reds_avg_pH_reset.merge(whites_avg_pH_reset) # Still doesn't work like this... why?

,quality,pH,alcohol


In [15]:
reds_avg_pH_reset.merge(whites_avg_pH_reset, on='quality')

,quality,pH_x,alcohol_x,pH_y,alcohol_y
0,3,3.398000,9.955000,3.187500,10.345000
1,4,3.381509,10.265094,3.182883,10.152454
2,5,3.304949,9.899706,3.168833,9.808840
3,6,3.318072,10.629519,3.188599,10.575372
4,7,3.290754,11.465913,3.213898,11.367936
5,8,3.267222,12.094444,3.218686,11.636000


In [16]:
# But I still want my high quality whites...?
reds_avg_pH_reset.merge(whites_avg_pH_reset, on='quality', how='right')

,quality,pH_x,alcohol_x,pH_y,alcohol_y
0,3,3.398000,9.955000,3.187500,10.345000
1,4,3.381509,10.265094,3.182883,10.152454
2,5,3.304949,9.899706,3.168833,9.808840
3,6,3.318072,10.629519,3.188599,10.575372
4,7,3.290754,11.465913,3.213898,11.367936
5,8,3.267222,12.094444,3.218686,11.636000
6,9,NaN,NaN,3.308000,12.180000


In [17]:
# And what are those x's, and y's, what about those?
reds_avg_pH_reset.merge(whites_avg_pH_reset, on='quality', 
                        how='right', suffixes=('_reds', '_whites'))

,quality,pH_reds,alcohol_reds,pH_whites,alcohol_whites
0,3,3.398000,9.955000,3.187500,10.345000
1,4,3.381509,10.265094,3.182883,10.152454
2,5,3.304949,9.899706,3.168833,9.808840
3,6,3.318072,10.629519,3.188599,10.575372
4,7,3.290754,11.465913,3.213898,11.367936
5,8,3.267222,12.094444,3.218686,11.636000
6,9,NaN,NaN,3.308000,12.180000


We can also do this by passing it into pd.merge, where the first argument is the DataFrame
that you would have called the `merge` method on (`reds_avg_pH_reset above`), and the second 
is the DataFrame you have passed into the `merge` method (`whites_avg_pH_reset_above`). 

In [18]:
pd.merge(reds_avg_pH_reset, whites_avg_pH_reset, on='quality', 
         how='right', suffixes=('_reds', '_whites'))

,quality,pH_reds,alcohol_reds,pH_whites,alcohol_whites
0,3,3.398000,9.955000,3.187500,10.345000
1,4,3.381509,10.265094,3.182883,10.152454
2,5,3.304949,9.899706,3.168833,9.808840
3,6,3.318072,10.629519,3.188599,10.575372
4,7,3.290754,11.465913,3.213898,11.367936
5,8,3.267222,12.094444,3.218686,11.636000
6,9,NaN,NaN,3.308000,12.180000


So, it turns out we could have also done this without resetting any of the indices. The `on` parameter in the `merge` calls will allow you to join on indices. 

In [19]:
reds_avg_pH.merge(whites_avg_pH, left_index=True, right_index=True, how='right', 
                  suffixes=('_reds', '_whites'))

,pH_reds,alcohol_reds,pH_whites,alcohol_whites
quality,,,,
3,3.398000,9.955000,3.187500,10.345000
4,3.381509,10.265094,3.182883,10.152454
5,3.304949,9.899706,3.168833,9.808840
6,3.318072,10.629519,3.188599,10.575372
7,3.290754,11.465913,3.213898,11.367936
8,3.267222,12.094444,3.218686,11.636000
9,NaN,NaN,3.308000,12.180000


We can even choose to join an index on a column...

In [20]:
reds_avg_pH.merge(whites_avg_pH_reset, left_index=True, right_on='quality',how='right', 
                  suffixes=('_reds', '_whites'))

,pH_reds,alcohol_reds,quality,pH_whites,alcohol_whites
0,3.398000,9.955000,3,3.187500,10.345000
1,3.381509,10.265094,4,3.182883,10.152454
2,3.304949,9.899706,5,3.168833,9.808840
3,3.318072,10.629519,6,3.188599,10.575372
4,3.290754,11.465913,7,3.213898,11.367936
5,3.267222,12.094444,8,3.218686,11.636000
6,NaN,NaN,9,3.308000,12.180000


## Final notes for `joining` and `merging`

Okay, so why these two methods? Well, there's not really a great answer. From my googling and blog post reading, it's not that one is substantially faster than the other (and we even noted that `join` calls `merge` on the back end), or anything like that. It kind of just comes down to convenience and programmer preference. Sorry for the disappointment...

![img](http://www.comicsus.com/wp-content/uploads/2013/06/sad-elephant-comicsus.png)

## Using `concat` (and `append`)

For maybe a little less of a bummer train, we will find that `concat` and `append` have slighty different use cases, and there is at least one clear-cut situation in which we **have** to use `concat` rather than `append`. `append` is only going to work if we want to tack rows on to the end of a `DataFrame`, and won't work if we want to tack extra columns on. `concat` will allow us to do either. 

### Concat

In [21]:
white_wine_df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6


So let's say we're running a machine learning algorithm, and we don't want `quality` to be put in as what is because the model will treat it as a continuous variable. We'll need some dummies. 

In [22]:
white_quality_dummies = pd.get_dummies(white_wine_df.quality, prefix='quality')

In [23]:
# Note we got back a DataFrame
print type(white_quality_dummies)

<class 'pandas.core.frame.DataFrame'>


In [24]:
white_quality_dummies.head()

,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9
0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0
2,0,0,0,1,0,0,0
3,0,0,0,1,0,0,0
4,0,0,0,1,0,0,0


So now we want to put these dummies as columns in our original DataFrame. Since the rows of this dummy DataFrame line up with the rows of the original DataFrame, we can just tack it on as if we were extending the DataFrame by these new columns. We'll see that `concat` will work here, while `append` won't. It's almost silly showing `append`, but it might be worthwhile to look at what it does do when you attempt this, so that if you see this in the future, you have an idea that this could be the reason. 

In [25]:
# Remember that this is called via the pandas module, and not directly on a DataFrame. 
pd.concat(white_wine_df, white_quality_dummies)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [26]:
# Oh, okay ^, so we have to pass in what we're concatenating as a list, where each item in 
# that list will be concatenated together. 
pd.concat([white_wine_df, white_quality_dummies]).head()

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,pH,quality,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,residual sugar,sulphates,total sulfur dioxide,volatile acidity
0,8.8,0.045,0.36,1.0010,7.0,45,3.00,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.7,0.45,170,0.27
1,9.5,0.049,0.34,0.9940,6.3,14,3.30,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.49,132,0.30
2,10.1,0.050,0.40,0.9951,8.1,30,3.26,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.9,0.44,97,0.28
3,9.9,0.058,0.32,0.9956,7.2,47,3.19,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5,0.40,186,0.23
4,9.9,0.058,0.32,0.9956,7.2,47,3.19,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5,0.40,186,0.23


In [27]:
# What happend here? Well, it looks like `concat` actually works like `append` by default, 
# and adds your two DataFrames by rows, rather than columns. That's an easy fix...
pd.concat([white_wine_df, white_quality_dummies], axis=1).head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6,0,0,0,1,0,0,0
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6,0,0,0,1,0,0,0
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6,0,0,0,1,0,0,0
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6,0,0,0,1,0,0,0
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6,0,0,0,1,0,0,0


### Append 

What happens if we use `append`? Since it works on rows, it'll look just like the `concat` call did when we didn't pass in the `axes` argument. 

In [28]:
# We're now back to calling these methods on a DataFrame, rather than the pandas module. 
white_wine_df.append(white_quality_dummies).head()

,alcohol,chlorides,citric acid,density,fixed acidity,free sulfur dioxide,pH,quality,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,residual sugar,sulphates,total sulfur dioxide,volatile acidity
0,8.8,0.045,0.36,1.0010,7.0,45,3.00,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.7,0.45,170,0.27
1,9.5,0.049,0.34,0.9940,6.3,14,3.30,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6,0.49,132,0.30
2,10.1,0.050,0.40,0.9951,8.1,30,3.26,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.9,0.44,97,0.28
3,9.9,0.058,0.32,0.9956,7.2,47,3.19,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5,0.40,186,0.23
4,9.9,0.058,0.32,0.9956,7.2,47,3.19,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.5,0.40,186,0.23


In [29]:
# Let's quickly look at appending on the rows. Let's take the first 5 rows, and then append
# the first five rows on in. 
first_five_wine_df = white_wine_df[0:5]
first_five_wine_df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6


In [30]:
first_five_wine_df.append(first_five_wine_df)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6


In [31]:
# Wait, so we have two sets of all the indices? This could cause problems in the future... is 
# there a way around fixing this? Yes!
first_five_wine_df.append(first_five_wine_df, ignore_index=True)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
5,7.0,0.27,0.36,20.7,0.045,45,170,1.0010,3.00,0.45,8.8,6
6,6.3,0.30,0.34,1.6,0.049,14,132,0.9940,3.30,0.49,9.5,6
7,8.1,0.28,0.40,6.9,0.050,30,97,0.9951,3.26,0.44,10.1,6
8,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6
9,7.2,0.23,0.32,8.5,0.058,47,186,0.9956,3.19,0.40,9.9,6


## The moral of our stories 

What did we end up learning today? Well, we learned that `merge` and `join` can be used interchangeably, and that `join` actually calls `merge` under the hood. We also learned that `append` and `concat` can be used somewhat interchangeably, but `concat` has to be used when we want to append two DataFrames along the column space, whereas either can be used when we want to append them along the row space. It turns out that `append` actually calls `concat` under the hood. 

While it might not seem like these distinctions are useful, I think that potentially the most important part of the day is just seeing how these work in order to recoginze potential errors, pitfalls, or unexpected results in your code, or other peoples code when you start using it. Of course you're code will always be perfect, but other people might not use these methods as intended, and when you end up staring at hundreds of rows of unexpected `NaNs` in a DataFrame that was created using somebody else's code, you'll now know a couple of more reasons as to how that could have happened. 

### Because why not?

![happy panda](http://o.aolcdn.com/dims5/amp:9c2b612dcbca544d17f9f7d3c8af65e9730ed8aa/r:960,504,min/c:960,504,0,3/q:80/?url=http%3A%2Fwww.blogcdn.com%2Fmassively.joystiq.com%2Fmedia%2F2012%2F10%2Fhappy-panda.jpg)